<a href="https://colab.research.google.com/github/2019zhou/flask-Couplet/blob/main/couplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
wget https://github.91chifun.workers.dev//https://github.com/pytorch/fairseq/archive/v0.10.1.zip

In [ ]:
%%bash
unzip v0.10.1.zip
mv fairseq-0.10.1 fairseq
cd fairseq
pip install --editable ./

In [ ]:
%%bash
wget -O "fairseq.tar.gz" https://lxylab.oss-cn-shanghai.aliyuncs.com/fairseq/fairseq.tar.gz?Expires=1613892173&OSSAccessKeyId=TMP.3KefFe833Phg9GYBdLccdwPsLocBjR6YnV9tbJMjYdo2Av1is9hgpgm9AvGeZufQXKvmaoMKEPPBBMBxTuQkcktbNQi2Cg&Signature=xRmS07NFI9XhPF9nc9AqxzW00oA%3D&versionId=CAEQGRiBgIDosIKNvhciIDZkZTM0M2EwZjViYjRjNjNhZTYxZjIwN2QxNjc3NDk4&response-content-type=application%2Foctet-stream
rm -rf ./fairseq
tar -zxvf "fairseq.tar.gz"
cd fairseq
pip install -e ./
cd ..

In [2]:
%%bash
mkdir -p /content/fairseq/checkpoints/fconv

In [3]:
%%bash
git clone https://github.com/luxinyu1/Chinese-couplet.git

Cloning into 'Chinese-couplet'...


In [ ]:
%%bash
wget https://lxylab.oss-cn-shanghai.aliyuncs.com/chinese-couplet/checkpoint_best.pt?Expires=1613749934&OSSAccessKeyId=TMP.3Kf8DcbtTrmbyK1db8w9NnGyN4Wt7c3SEkAGL3v2UDzkc2CRusmiWLEjdSBVPatN4N7mEDcxBHipFrSxkfvKRA11ptXbYn&Signature=rNsVORC2gK5ioKszqsWN7kcQ0uU%3D&versionId=CAEQGRiBgMCl1vXfvRciIGJhNTkyYTU0NzdhNTRmNDA5ZjY3ZjIzNWI2NDQzNTNi&response-content-type=application%2Foctet-stream

In [5]:
%%bash
mv checkpoint_best.pt?Expires=1613749934 checkpoint_best.pt

In [6]:
!mv checkpoint_best.pt /content/fairseq/checkpoints/fconv/

In [7]:
%cd fairseq

/content/fairseq


In [8]:
mv /content/Chinese-couplet/couplet-bin /content/fairseq

In [ ]:
%%bash
fairseq-generate /content/couplet-bin \
  --source-lang "src" \
  --target-lang "dst" \
  --path /content/checkpoint_best.pt \
  --beam 5 \

In [16]:
%%bash
fairseq-interactive /content/fairseq/couplet-bin \
    --path /content/fairseq/checkpoints/fconv/checkpoint_best.pt \
    --source-lang "src" \
    --target-lang "dst" \
    --beam 5 --input /content/input.txt >  /content/output.txt

In [25]:
%%bash
grep ^H /content/output.txt | cut -f3- > /content/output2.txt

In [32]:
!pip install Flask

In [33]:
!pip install flask-ngrok

In [44]:
import subprocess
import re 

def getresult(firstLine):
  print(firstLine)
  firstLine = firstLine.strip()
  strtmp = ""
  for x in firstLine:
    strtmp += x + " "
  filename = "/content/input.txt"
  file = open(filename, "w")
  file.write(strtmp)
  file.close()
  p = subprocess.Popen(['fairseq-interactive /content/fairseq/couplet-bin \
  --path /content/fairseq/checkpoints/fconv/checkpoint_best.pt --source-lang "src" \
  --target-lang "dst" --beam 5 --input %s > /content/output.txt'%(filename)],shell=True, stdout=subprocess.PIPE)  
  out, err = p.communicate()  
  p2 = subprocess.Popen(['grep ^H /content/output.txt | cut -f3- > /content/output2.txt'],shell=True)
  out2, err2 = p2.communicate()
  f = open("/content/output2.txt")
  res = f.read()
  f.close()
  return "".join(res.split())


In [45]:
getresult("春眠不知晓")

春眠不知晓


'秋雨不知音'

In [37]:
%cd /content

/content


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

# def getresult(firstLine):
#     p = subprocess.Popen('fairseq-interactive couplet-bin --source-lang "src" --target-lang "dst" \
#     --path checkpoint_best.pt \
#     --beam 5 ',shell=True,stdout=subprocess.PIPE)  
#     out,err = p.communicate()  
#     line1 = ""
#     print(out)
#     for line in out.splitlines():
#         print(line)
#         line1 = line
#     return line1


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    elif request.method == 'POST':
        secondlinetmp = getresult(request.form['firstLine'])
        Couplets = {
            'firstLine':request.form['firstLine'],
            'secondLine': secondlinetmp,
        }
        return render_template('predict.html', **Couplets)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://17846882dd86.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Feb/2021 07:51:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2021 07:51:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Feb/2021 07:51:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2021 07:51:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2021 07:51:50] "GET /favicon.ico HTTP/1.1" 404 -


春 眠 不 觉 晓


127.0.0.1 - - [21/Feb/2021 07:52:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2021 07:52:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2021 07:52:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2021 07:52:41] "GET / HTTP/1.1" 200 -


春眠不知晓


127.0.0.1 - - [21/Feb/2021 07:55:47] "POST / HTTP/1.1" 200 -
